In [ ]:
#Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
id_test = test.id

y_train = train["price_doc"]
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)

for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values)) 
        x_train[c] = lbl.transform(list(x_train[c].values))
        
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))

xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

In [ ]:
#Adversarial validation
xtrain = x_train
xtest = x_test

xtrain.fillna(-1,inplace = True)
xtest.fillna(-1, inplace = True)

In [ ]:
# add identifier and combine
xtrain['istrain'] = 1
xtest['istrain'] = 0
xdat = pd.concat([xtrain, xtest], axis = 0)

# convert non-numerical columns to integers
df_numeric = xdat.select_dtypes(exclude=['object'])
df_obj = xdat.select_dtypes(include=['object']).copy()
    
for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]
    
xdat = pd.concat([df_numeric, df_obj], axis=1)
y = xdat['istrain']; xdat.drop('istrain', axis = 1, inplace = True)

In [ ]:
x_train = x_train.drop('istrain',axis=1)
x_test = x_test.drop('istrain',axis=1)

#use XGboost to predict
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 44)
xgb_params = {
        'learning_rate': 0.05, 'max_depth': 4,'subsample': 0.9,
        'colsample_bytree': 0.9,'objective': 'binary:logistic',
        'silent': 1, 'n_estimators':100, 'gamma':1,
        'min_child_weight':4
        }   
clf = xgb.XGBClassifier(**xgb_params, seed = 10)  

In [ ]:
for train_index, test_index in skf.split(xdat, y):
        x0, x1 = xdat.iloc[train_index], xdat.iloc[test_index]
        y0, y1 = y.iloc[train_index], y.iloc[test_index]        
        print(x0.shape)
        clf.fit(x0, y0, eval_set=[(x1, y1)],
               eval_metric='logloss', verbose=False,early_stopping_rounds=10)
                
        prval = clf.predict_proba(x1)[:,1]
        print(roc_auc_score(y1,prval))

In [ ]:
rt = xdat.iloc[train_index]
rt['y'] = clf.predict_proba(rt)[:,1]

#set validation set as 0.3
dev = rt.sort(['y'],ascending = 0).head(21277)
dev_index = dev.index

In [ ]:
val_X = x_train[~x_train.index.isin(dev_index)]
val_y = y_train[~y_train.index.isin(dev_index)]

dev_X = x_train.ix[dev_index]
dev_y = y_train.ix[dev_index]

In [ ]:
xgtrain = xgb.DMatrix(dev_X, dev_y, feature_names=dev_X.columns)
xgtest = xgb.DMatrix(val_X, val_y, feature_names=val_X.columns)
watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]


num_boost_rounds = 384  # This was the CV output, as earlier version shows
#model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)
model = xgb.train(xgb_params, xgtrain, num_boost_rounds, watchlist, early_stopping_rounds=20, verbose_eval=5)

In [ ]:
dtest = xgb.DMatrix(x_test)
y_predict = model.predict(dtest)

op2 = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
op2.head()